# Ablation study of our method
Checking if the random events and clustering/grouping improves our method at all

In [1]:
import sys
import os
import pandas as pd
import numpy as np
import generative_model
import matplotlib.pyplot as plt
from tqdm import tqdm
import csv
np.random.seed(0)

## Helper functions

In [2]:
def calculate_FDE(pred_x, pred_y, test_x, test_y):

    final_displacement_x = pred_x[-1] - test_x[-1]
    final_displacement_y = pred_y[-1] - test_y[-1]
    FDE = np.sqrt(final_displacement_x**2 + final_displacement_y**2)
    
    return FDE

def calculate_ADE(pred_x, pred_y, test_x, test_y):
    assert len(pred_x) == len(test_x)
    total_displacement_error = 0
    for point_idx in range(len(test_x)):
        displacement_error = np.sqrt((pred_x[point_idx] - test_x[point_idx])**2 + (pred_y[point_idx] - test_y[point_idx])**2)
        total_displacement_error += displacement_error

    return total_displacement_error/len(pred_x)

## The evaluation logic for Trajectron++ loops over the frames and predicts the future trajectories 
## for each node present in the current frame
## Each node has to have at least 7 historical points and 12 future points
def get_total_predictable_slices(data):
    total_predictable_steps = 0
    for i in pd.unique(data.node_id):
        #print(len(test[test.node_id == i]))
        total_predictable_steps += len(data[data.node_id == i]) - 19
    return total_predictable_steps

In [3]:
def process_data(input_data):
    data = input_data.copy()
    data['frame_id'] = pd.to_numeric(data['frame_id'], downcast='integer')
    data['track_id'] = pd.to_numeric(data['track_id'], downcast='integer')

    data['frame_id'] = data['frame_id'] // 10

    data['frame_id'] -= data['frame_id'].min()

    data['node_type'] = 'PEDESTRIAN'
    data['node_id'] = data['track_id'].astype(str)
    data.sort_values('frame_id', inplace=True)

    data['pos_x'] = data['pos_x'] - data['pos_x'].mean()
    data['pos_y'] = data['pos_y'] - data['pos_y'].mean()
    
    # Select only such nodes which have enough data to predict on (8 historical timesteps, 12 future)
    v = data.node_id.value_counts()
    data = data[data.node_id.isin(v.index[v.gt(19)])]
    
    return data

## Method evaluation logic

In [4]:
def evaluate_our_method(data, params, dataset_title='', clustering_method='KMeans', smoothing=False):
    our_fde_best_of = []
    our_ade_best_of = []
    our_fde_single = []
    our_ade_single = []

    # Loop over the dataset frame by frame
    for frame_id in tqdm(pd.unique(data.frame_id), desc='Ours - ' + dataset_title):

        frame_data = data[data.frame_id == frame_id]
        
        # Loop over all agents in the current frame
        for node_id in pd.unique(frame_data.node_id):
            # Check for 8 points of history for current agent
            if len(data[((data.node_id == node_id) & (data.frame_id <= frame_id))]) >= 8:
                # Check for 12 points of future for current agent
                if len(data[((data.node_id == node_id) & (data.frame_id > frame_id))]) >= 12:
                    node_history_data = data[((data.node_id == node_id) & (data.frame_id <= frame_id) & (data.frame_id >= frame_id-7))]
                    node_gt_data = data[((data.node_id == node_id) & (data.frame_id > frame_id) & (data.frame_id <= frame_id+12))]

                    x_data = list(node_history_data.pos_x)
                    y_data = list(node_history_data.pos_y)
                    assert len(x_data) == 8

                    x_gt = list(node_gt_data.pos_x)
                    y_gt = list(node_gt_data.pos_y)
                    assert len(x_gt) == 12

                    all_pred_x, all_pred_y, weights = generative_model.predict(x_data, y_data, params, trajectory_length=12, clustering_method=clustering_method, smoothing=smoothing)
                    assert len(all_pred_x[0]) == 12
                    
                    # This section is for producing a single trajectory (averaging all representative preds)
                    avg_x = np.average(all_pred_x, axis=0, weights=weights)
                    avg_y = np.average(all_pred_y, axis=0, weights=weights)
                    #avg_x = np.mean(all_pred_x, axis=0)
                    #avg_y = np.mean(all_pred_y, axis=0)
                    avg_fde = calculate_FDE(avg_x, avg_y, x_gt, y_gt)
                    avg_ade = calculate_ADE(avg_x, avg_y, x_gt, y_gt)
                    our_fde_single.append(avg_fde)
                    our_ade_single.append(avg_ade)
                        
                        
                    # This section is for finding the best trajectories out of many in terms of FDE and ADE
                    best_fde = None
                    best_ade = None

                    for i in range(len(all_pred_x)):
                        current_pred_x = all_pred_x[i]
                        current_pred_y = all_pred_y[i]

                        fde = calculate_FDE(current_pred_x, current_pred_y, x_gt, y_gt)
                        if best_fde == None or fde < best_fde:
                            best_fde = fde

                        ade = calculate_ADE(current_pred_x, current_pred_y, x_gt, y_gt)
                        if best_ade == None or ade < best_ade:
                            best_ade = ade

                    our_fde_best_of.append(best_fde)
                    our_ade_best_of.append(best_ade)
                    
    return our_fde_best_of, our_ade_best_of, our_fde_single, our_ade_single


## Automated results comparison

In [6]:
def read_trajectron_data(trajectron_resultset_name, base_folder='./trajectron++/results_paper_version/', suffix='best_of'):
    trajectron_fde = []
    with open(base_folder + trajectron_resultset_name + '_fde_' + suffix + '.csv', mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            trajectron_fde.append(float(row['value']))

    trajectron_ade = []
    with open(base_folder + trajectron_resultset_name + '_ade_'+ suffix + '.csv', mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            trajectron_ade.append(float(row['value']))
            
    return trajectron_fde, trajectron_ade

In [19]:
def evaluate_all_datasets(our_method_params, datasets, trajectron_resultset_names, trajectron_ar3_resultset_names, evaluate_most_likely=False, evaluate_with_interactions=False):
    base_path = './raw_data/'

    ours_results = {'BEST_OF_20': {'FDE': [], 'ADE': []}, 'MOST_LIKELY': {'FDE': [], 'ADE': []}}
    trajectron_results = {'BEST_OF_20': {'FDE': [], 'ADE': []}, 'MOST_LIKELY': {'FDE': [], 'ADE': []}}
    trajectron_ar3_results = {'BEST_OF_20': {'FDE': [], 'ADE': []}, 'MOST_LIKELY': {'FDE': [], 'ADE': []}}

    for dataset_idx, dataset in enumerate(datasets):
        our_fde_bo20, our_ade_bo20 = [], []
        our_fde_most_likely, our_ade_most_likely = [], []
        
        for scene_idx, scene in enumerate(dataset):
            data = pd.read_csv(base_path + scene, sep='\t', index_col=False, header=None)
            data.columns = ['frame_id', 'track_id', 'pos_x', 'pos_y']

            data = process_data(data)

            ## Ours
            our_fde_best_of_20, our_ade_best_of_20, our_fde_single, our_ade_single = evaluate_our_method(data, our_method_params[dataset_idx], dataset_title=trajectron_resultset_names[dataset_idx], smoothing=True)
            our_fde_bo20 += our_fde_best_of_20
            our_ade_bo20 += our_ade_best_of_20
            if evaluate_most_likely:
                our_fde_most_likely += our_fde_single
                our_ade_most_likely += our_ade_single

        # add our results to the data dict
        ours_results['BEST_OF_20']['FDE'].append(np.mean(our_fde_bo20))
        ours_results['BEST_OF_20']['ADE'].append(np.mean(our_ade_bo20))
        if evaluate_most_likely:
            ours_results['MOST_LIKELY']['FDE'].append(np.mean(our_fde_most_likely))
            ours_results['MOST_LIKELY']['ADE'].append(np.mean(our_ade_most_likely))

                
        ## Trajectron
        trajectron_fde, trajectron_ade = read_trajectron_data(trajectron_resultset_names[dataset_idx])
        trajectron_results['BEST_OF_20']['FDE'].append(np.mean(trajectron_fde))
        trajectron_results['BEST_OF_20']['ADE'].append(np.mean(trajectron_ade))
        
        if evaluate_most_likely:
            trajectron_fde, trajectron_ade = read_trajectron_data(trajectron_resultset_names[dataset_idx], suffix='most_likely')
            trajectron_results['MOST_LIKELY']['FDE'].append(np.mean(trajectron_fde))
            trajectron_results['MOST_LIKELY']['ADE'].append(np.mean(trajectron_ade))
        
        ## Trajectron with interactions
        if evaluate_with_interactions:
            trajectron_ar3_fde, trajectron_ar3_ade = read_trajectron_data(trajectron_ar3_resultset_names[dataset_idx])
            trajectron_ar3_results['BEST_OF_20']['FDE'].append(np.mean(trajectron_ar3_fde))
            trajectron_ar3_results['BEST_OF_20']['ADE'].append(np.mean(trajectron_ar3_ade))

            if evaluate_most_likely:
                trajectron_ar3_fde, trajectron_ar3_ade = read_trajectron_data(trajectron_resultset_names[dataset_idx], suffix='most_likely')
                trajectron_ar3_results['MOST_LIKELY']['FDE'].append(np.mean(trajectron_ar3_fde))
                trajectron_ar3_results['MOST_LIKELY']['ADE'].append(np.mean(trajectron_ar3_ade))

        # make sure that there is no discrepancy between our data processing and trajectron evaluation results size
        if len(dataset) == 1:
            num_predictable_trajectories = get_total_predictable_slices(data)
            assert len(trajectron_fde) == num_predictable_trajectories
            assert len(trajectron_ade) == num_predictable_trajectories
    
    return [
        ours_results,
        trajectron_results,
        trajectron_ar3_results,
    ]

### Running the evaluation

In [16]:
datasets = [
    ['eth/test/biwi_eth.txt'], 
    ['hotel/test/biwi_hotel.txt'], 
    ['univ/test/students001.txt', 'univ/test/students003.txt'],
    ['zara1/test/crowds_zara01.txt'], 
    ['zara2/test/crowds_zara02.txt'],
]

trajectron_resultset_names = [
    'eth_vel', 
    'hotel_vel',
    'univ_vel',
    'zara1_vel_modified', 
    'zara2_vel_modified',
]

trajectron_ar3_resultset_names = [
    'eth_ar3', 
    'hotel_ar3', 
    'univ_ar3',
    'zara1_ar3', 
    'zara2_ar3',
]

In [ ]:
no_events_no_grouping_params = {
    'NOISE': 0.05, 
    'NO_OF_TRAJECTORIES': 300, 
    'CONST_VEL_MODEL_PROB': 1.0, 
    'STOP_PROB': 0.0, 
    'DISCOUNT_AVG_PROB': 1, 
    'DISCOUNT_LOWER_BOUND': 0.2, 
    'VELOCITY_CHANGE_PROB': 0.0,
    'VELOCITY_CHANGE_NOISE': 0.1, 
    'ANGLE_CHANGE_PROB': 0.0, 
    'ANGLE_CHANGE_NOISE': 2.5, 
    'GROUP_PERCENTAGES': [1.0], 
    'GROUP_CLUSTER_COUNT': [20]
}


#our_method_params = [eth_params, hotel_params, univ_params, zara1_params, zara2_params]
our_method_params = [no_events_no_grouping_params, 
                     no_events_no_grouping_params, 
                     no_events_no_grouping_params, 
                     no_events_no_grouping_params, 
                     no_events_no_grouping_params]

no_event_no_grouping_res = evaluate_all_datasets(our_method_params, datasets, trajectron_resultset_names, trajectron_ar3_resultset_names, evaluate_most_likely=True)


In [21]:
velocity_event_no_grouping_params = {
    'NOISE': 0.05, 
    'NO_OF_TRAJECTORIES': 300, 
    'CONST_VEL_MODEL_PROB': 1.0, 
    'STOP_PROB': 0.0, 
    'DISCOUNT_AVG_PROB': 1, 
    'DISCOUNT_LOWER_BOUND': 0.2, 
    'VELOCITY_CHANGE_PROB': 0.2,
    'VELOCITY_CHANGE_NOISE': 0.1, 
    'ANGLE_CHANGE_PROB': 0.0, 
    'ANGLE_CHANGE_NOISE': 2.5, 
    'GROUP_PERCENTAGES': [1.0], 
    'GROUP_CLUSTER_COUNT': [20]
}


#our_method_params = [eth_params, hotel_params, univ_params, zara1_params, zara2_params]
our_method_params = [velocity_event_no_grouping_params, 
                     velocity_event_no_grouping_params, 
                     velocity_event_no_grouping_params, 
                     velocity_event_no_grouping_params, 
                     velocity_event_no_grouping_params]

velocity_event_no_grouping_res = evaluate_all_datasets(our_method_params, datasets, trajectron_resultset_names, trajectron_ar3_resultset_names, evaluate_most_likely=True)


Ours - zara2_vel_modified: 100%|████████████████████████████████████████████████████████████| 1052/1052 [06:21<00:00,  2.76it/s]


In [22]:
angle_event_no_grouping_params = {
    'NOISE': 0.05, 
    'NO_OF_TRAJECTORIES': 300, 
    'CONST_VEL_MODEL_PROB': 1.0, 
    'STOP_PROB': 0.00, 
    'DISCOUNT_AVG_PROB': 1, 
    'DISCOUNT_LOWER_BOUND': 0.2, 
    'VELOCITY_CHANGE_PROB': 0.0,
    'VELOCITY_CHANGE_NOISE': 0.1, 
    'ANGLE_CHANGE_PROB': 0.2, 
    'ANGLE_CHANGE_NOISE': 2.5, 
    'GROUP_PERCENTAGES': [1.0], 
    'GROUP_CLUSTER_COUNT': [20]
}


#our_method_params = [eth_params, hotel_params, univ_params, zara1_params, zara2_params]
our_method_params = [angle_event_no_grouping_params, 
                     angle_event_no_grouping_params, 
                     angle_event_no_grouping_params, 
                     angle_event_no_grouping_params, 
                     angle_event_no_grouping_params]

angle_event_no_grouping_res = evaluate_all_datasets(our_method_params, datasets, trajectron_resultset_names, trajectron_ar3_resultset_names, evaluate_most_likely=True)


Ours - zara2_vel_modified: 100%|████████████████████████████████████████████████████████████| 1052/1052 [06:49<00:00,  2.57it/s]


In [23]:
angle_and_velocity_event_no_grouping_params = {
    'NOISE': 0.05, 
    'NO_OF_TRAJECTORIES': 300, 
    'CONST_VEL_MODEL_PROB': 1.0, 
    'STOP_PROB': 0.0, 
    'DISCOUNT_AVG_PROB': 1, 
    'DISCOUNT_LOWER_BOUND': 0.2, 
    'VELOCITY_CHANGE_PROB': 0.2,
    'VELOCITY_CHANGE_NOISE': 0.1, 
    'ANGLE_CHANGE_PROB': 0.2, 
    'ANGLE_CHANGE_NOISE': 2.5, 
    'GROUP_PERCENTAGES': [1.0], 
    'GROUP_CLUSTER_COUNT': [20]
}


#our_method_params = [eth_params, hotel_params, univ_params, zara1_params, zara2_params]
our_method_params = [angle_and_velocity_event_no_grouping_params, 
                     angle_and_velocity_event_no_grouping_params, 
                     angle_and_velocity_event_no_grouping_params, 
                     angle_and_velocity_event_no_grouping_params, 
                     angle_and_velocity_event_no_grouping_params]

angle_and_velocity_event_no_grouping_res = evaluate_all_datasets(our_method_params, datasets, trajectron_resultset_names, trajectron_ar3_resultset_names, evaluate_most_likely=True)


Ours - zara2_vel_modified: 100%|████████████████████████████████████████████████████████████| 1052/1052 [06:59<00:00,  2.51it/s]


In [24]:
no_event_grouping_params = {
    'NOISE': 0.05, 
    'NO_OF_TRAJECTORIES': 300, 
    'CONST_VEL_MODEL_PROB': 1.0, 
    'STOP_PROB': 0.0, 
    'DISCOUNT_AVG_PROB': 1, 
    'DISCOUNT_LOWER_BOUND': 0.2, 
    'VELOCITY_CHANGE_PROB': 0.0,
    'VELOCITY_CHANGE_NOISE': 0.1, 
    'ANGLE_CHANGE_PROB': 0.0, 
    'ANGLE_CHANGE_NOISE': 2.5, 
    'GROUP_PERCENTAGES': [0.1, 0.4, 0.65, 0.85, 1.0], 
    'GROUP_CLUSTER_COUNT': [1, 6, 5, 5, 3]
}


#our_method_params = [eth_params, hotel_params, univ_params, zara1_params, zara2_params]
our_method_params = [no_event_grouping_params, 
                     no_event_grouping_params, 
                     no_event_grouping_params, 
                     no_event_grouping_params, 
                     no_event_grouping_params]

no_event_grouping_res = evaluate_all_datasets(our_method_params, datasets, trajectron_resultset_names, trajectron_ar3_resultset_names, evaluate_most_likely=True)

Ours - zara2_vel_modified: 100%|████████████████████████████████████████████████████████████| 1052/1052 [07:16<00:00,  2.41it/s]


In [25]:
velocity_event_grouping_params = {
    'NOISE': 0.05, 
    'NO_OF_TRAJECTORIES': 300, 
    'CONST_VEL_MODEL_PROB': 1.0, 
    'STOP_PROB': 0.0, 
    'DISCOUNT_AVG_PROB': 1, 
    'DISCOUNT_LOWER_BOUND': 0.2, 
    'VELOCITY_CHANGE_PROB': 0.2,
    'VELOCITY_CHANGE_NOISE': 0.1, 
    'ANGLE_CHANGE_PROB': 0., 
    'ANGLE_CHANGE_NOISE': 2.5, 
    'GROUP_PERCENTAGES': [0.1, 0.4, 0.65, 0.85, 1.0], 
    'GROUP_CLUSTER_COUNT': [1, 6, 5, 5, 3]
}


#our_method_params = [eth_params, hotel_params, univ_params, zara1_params, zara2_params]
our_method_params = [velocity_event_grouping_params, 
                     velocity_event_grouping_params, 
                     velocity_event_grouping_params, 
                     velocity_event_grouping_params, 
                     velocity_event_grouping_params]

velocity_event_grouping_res = evaluate_all_datasets(our_method_params, datasets, trajectron_resultset_names, trajectron_ar3_resultset_names, evaluate_most_likely=True)

Ours - zara2_vel_modified: 100%|████████████████████████████████████████████████████████████| 1052/1052 [06:01<00:00,  2.91it/s]


In [26]:
angle_event_grouping_params = {
    'NOISE': 0.05, 
    'NO_OF_TRAJECTORIES': 300, 
    'CONST_VEL_MODEL_PROB': 1.0, 
    'STOP_PROB': 0.0, 
    'DISCOUNT_AVG_PROB': 1, 
    'DISCOUNT_LOWER_BOUND': 0.2, 
    'VELOCITY_CHANGE_PROB': 0.0,
    'VELOCITY_CHANGE_NOISE': 0.1, 
    'ANGLE_CHANGE_PROB': 0.2, 
    'ANGLE_CHANGE_NOISE': 2.5, 
    'GROUP_PERCENTAGES': [0.1, 0.4, 0.65, 0.85, 1.0], 
    'GROUP_CLUSTER_COUNT': [1, 6, 5, 5, 3]
}


#our_method_params = [eth_params, hotel_params, univ_params, zara1_params, zara2_params]
our_method_params = [angle_event_grouping_params, 
                     angle_event_grouping_params, 
                     angle_event_grouping_params, 
                     angle_event_grouping_params, 
                     angle_event_grouping_params]

angle_event_grouping_res = evaluate_all_datasets(our_method_params, datasets, trajectron_resultset_names, trajectron_ar3_resultset_names, evaluate_most_likely=True)

Ours - zara2_vel_modified: 100%|████████████████████████████████████████████████████████████| 1052/1052 [05:58<00:00,  2.94it/s]


In [27]:
angle_and_velocity_event_grouping_params = {
    'NOISE': 0.05, 
    'NO_OF_TRAJECTORIES': 300, 
    'CONST_VEL_MODEL_PROB': 1.0, 
    'STOP_PROB': 0.0, 
    'DISCOUNT_AVG_PROB': 1, 
    'DISCOUNT_LOWER_BOUND': 0.2, 
    'VELOCITY_CHANGE_PROB': 0.2,
    'VELOCITY_CHANGE_NOISE': 0.1, 
    'ANGLE_CHANGE_PROB': 0.2, 
    'ANGLE_CHANGE_NOISE': 2.5, 
    'GROUP_PERCENTAGES': [0.1, 0.4, 0.65, 0.85, 1.0], 
    'GROUP_CLUSTER_COUNT': [1, 6, 5, 5, 3]
}


#our_method_params = [eth_params, hotel_params, univ_params, zara1_params, zara2_params]
our_method_params = [angle_and_velocity_event_grouping_params, 
                     angle_and_velocity_event_grouping_params, 
                     angle_and_velocity_event_grouping_params, 
                     angle_and_velocity_event_grouping_params, 
                     angle_and_velocity_event_grouping_params]

angle_and_velocity_event_grouping_res = evaluate_all_datasets(our_method_params, datasets, trajectron_resultset_names, trajectron_ar3_resultset_names, evaluate_most_likely=True)

Ours - zara2_vel_modified: 100%|████████████████████████████████████████████████████████████| 1052/1052 [06:06<00:00,  2.87it/s]


In [35]:
ours_results = ours_results_no_events
index = [
    'ETH', 
    'Hotel', 
    'Univ',
    'Zara 1', 
    'Zara 2',
    
]

df_data_best_of_20_fde = {
    'CVM (8pt history)' : pd.Series(cvm_long_results['BEST_OF_20']['FDE'], index = index),
    'CVM (2pt history)' : pd.Series(cvm_short_results['BEST_OF_20']['FDE'], index = index),
    'Social-GAN': pd.Series(sgan_results['BEST_OF_20']['FDE'], index = index),
    'Trajectron++' : pd.Series(trajectron_results['BEST_OF_20']['FDE'], index = index),
    #'Trajectron++ AR3' : pd.Series(trajectron_ar3_results['BEST_OF_20']['FDE'], index = index),
    'Ours': pd.Series(ours_results['BEST_OF_20']['FDE'], index = index)
}

df_best_of_20_fde = pd.DataFrame(df_data_best_of_20_fde)

df_data_best_of_20_ade = {
    'CVM (8pt history)' : pd.Series(cvm_long_results['BEST_OF_20']['ADE'], index = index),
    'CVM (2pt history)' : pd.Series(cvm_short_results['BEST_OF_20']['ADE'], index = index),
    'Social-GAN': pd.Series(sgan_results['BEST_OF_20']['ADE'], index = index),
    'Trajectron++' : pd.Series(trajectron_results['BEST_OF_20']['ADE'], index = index),
    #'Trajectron++ AR3' : pd.Series(trajectron_ar3_results['BEST_OF_20']['ADE'], index = index),
    'Ours': pd.Series(ours_results['BEST_OF_20']['ADE'], index = index)
}

df_best_of_20_ade = pd.DataFrame(df_data_best_of_20_ade)

df_data_most_likely_fde = {
    'CVM (8pt history)' : pd.Series(cvm_long_results['MOST_LIKELY']['FDE'], index = index),
    'CVM (2pt history)' : pd.Series(cvm_short_results['MOST_LIKELY']['FDE'], index = index),
    'Social-GAN': pd.Series(sgan_results['MOST_LIKELY']['FDE'], index = index),
    'Trajectron++' : pd.Series(trajectron_results['MOST_LIKELY']['FDE'], index = index),
    #'Trajectron++ AR3' : pd.Series(trajectron_ar3_results['MOST_LIKELY']['FDE'], index = index),
    'Ours': pd.Series(ours_results['MOST_LIKELY']['FDE'], index = index)
}

df_most_likely_fde = pd.DataFrame(df_data_most_likely_fde)

df_data_most_likely_ade = {
    'CVM (8pt history)' : pd.Series(cvm_long_results['MOST_LIKELY']['ADE'], index = index),
    'CVM (2pt history)' : pd.Series(cvm_short_results['MOST_LIKELY']['ADE'], index = index),
    'Social-GAN': pd.Series(sgan_results['MOST_LIKELY']['ADE'], index = index),
    'Trajectron++' : pd.Series(trajectron_results['MOST_LIKELY']['ADE'], index = index),
    #'Trajectron++ AR3' : pd.Series(trajectron_ar3_results['MOST_LIKELY']['ADE'], index = index),
    'Ours': pd.Series(ours_results['MOST_LIKELY']['ADE'], index = index)
}

df_most_likely_ade = pd.DataFrame(df_data_most_likely_ade)

In [44]:
index = [
    'ETH', 
    'Hotel', 
    'Univ',
    'Zara 1', 
    'Zara 2',
    
]

df_data_best_of_20_fde = {
    'No events, no grouping' : pd.Series(no_event_no_grouping_res[0]['BEST_OF_20']['FDE'], index = index),
    'Velocity event, no grouping' : pd.Series(velocity_event_no_grouping_res[0]['BEST_OF_20']['FDE'], index = index),
    'Angle event, no grouping' : pd.Series(angle_event_no_grouping_res[0]['BEST_OF_20']['FDE'], index = index),
    'Angle + velocity event, no grouping' : pd.Series(angle_and_velocity_event_no_grouping_res[0]['BEST_OF_20']['FDE'], index = index),
    'No events, with grouping' : pd.Series(no_event_grouping_res[0]['BEST_OF_20']['FDE'], index = index),
    'Velocity event, with grouping' : pd.Series(velocity_event_grouping_res[0]['BEST_OF_20']['FDE'], index = index),
    'Angle event, with grouping' : pd.Series(angle_event_grouping_res[0]['BEST_OF_20']['FDE'], index = index),
    'Angle + velocity event, with grouping' : pd.Series(angle_and_velocity_event_grouping_res[0]['BEST_OF_20']['FDE'], index = index),
}
df_best_of_20_fde = pd.DataFrame(df_data_best_of_20_fde)

df_data_best_of_20_ade = {
    'No events, no grouping' : pd.Series(no_event_no_grouping_res[0]['BEST_OF_20']['ADE'], index = index),
    'Velocity event, no grouping' : pd.Series(velocity_event_no_grouping_res[0]['BEST_OF_20']['ADE'], index = index),
    'Angle event, no grouping' : pd.Series(angle_event_no_grouping_res[0]['BEST_OF_20']['ADE'], index = index),
    'Angle + velocity event, no grouping' : pd.Series(angle_and_velocity_event_no_grouping_res[0]['BEST_OF_20']['ADE'], index = index),
    'No events, with grouping' : pd.Series(no_event_grouping_res[0]['BEST_OF_20']['ADE'], index = index),
    'Velocity event, with grouping' : pd.Series(velocity_event_grouping_res[0]['BEST_OF_20']['ADE'], index = index),
    'Angle event, with grouping' : pd.Series(angle_event_grouping_res[0]['BEST_OF_20']['ADE'], index = index),
    'Angle + velocity event, with grouping' : pd.Series(angle_and_velocity_event_grouping_res[0]['BEST_OF_20']['ADE'], index = index),
}
df_best_of_20_ade = pd.DataFrame(df_data_best_of_20_ade)

df_data_single_fde = {
    'No events, no grouping' : pd.Series(no_event_no_grouping_res[0]['MOST_LIKELY']['FDE'], index = index),
    'Velocity event, no grouping' : pd.Series(velocity_event_no_grouping_res[0]['MOST_LIKELY']['FDE'], index = index),
    'Angle event, no grouping' : pd.Series(angle_event_no_grouping_res[0]['MOST_LIKELY']['FDE'], index = index),
    'Angle + velocity event, no grouping' : pd.Series(angle_and_velocity_event_no_grouping_res[0]['MOST_LIKELY']['FDE'], index = index),
    'No events, with grouping' : pd.Series(no_event_grouping_res[0]['MOST_LIKELY']['FDE'], index = index),
    'Velocity event, with grouping' : pd.Series(velocity_event_grouping_res[0]['MOST_LIKELY']['FDE'], index = index),
    'Angle event, with grouping' : pd.Series(angle_event_grouping_res[0]['MOST_LIKELY']['FDE'], index = index),
    'Angle + velocity event, with grouping' : pd.Series(angle_and_velocity_event_grouping_res[0]['MOST_LIKELY']['FDE'], index = index),
}
df_single_fde = pd.DataFrame(df_data_single_fde)

df_data_single_ade = {
    'No events, no grouping' : pd.Series(no_event_no_grouping_res[0]['MOST_LIKELY']['ADE'], index = index),
    'Velocity event, no grouping' : pd.Series(velocity_event_no_grouping_res[0]['MOST_LIKELY']['ADE'], index = index),
    'Angle event, no grouping' : pd.Series(angle_event_no_grouping_res[0]['MOST_LIKELY']['ADE'], index = index),
    'Angle + velocity event, no grouping' : pd.Series(angle_and_velocity_event_no_grouping_res[0]['MOST_LIKELY']['ADE'], index = index),
    'No events, with grouping' : pd.Series(no_event_grouping_res[0]['MOST_LIKELY']['ADE'], index = index),
    'Velocity event, with grouping' : pd.Series(velocity_event_grouping_res[0]['MOST_LIKELY']['ADE'], index = index),
    'Angle event, with grouping' : pd.Series(angle_event_grouping_res[0]['MOST_LIKELY']['ADE'], index = index),
    'Angle + velocity event, with grouping' : pd.Series(angle_and_velocity_event_grouping_res[0]['MOST_LIKELY']['ADE'], index = index),
}
df_single_ade = pd.DataFrame(df_data_single_ade)


In [39]:
df_best_of_20_fde.style.highlight_min(color = 'lightgreen', axis = 1)

,"No events, no grouping","Velocity event, no grouping","Angle event, no grouping","Angle + velocity event, no grouping","No events, with grouping","Velocity event, with grouping","Angle event, with grouping","Angle + velocity event, with grouping"
ETH,1.387284,0.849455,1.392594,0.853919,1.554313,1.034541,1.551465,1.039120
Hotel,0.221257,0.317681,0.223753,0.322534,0.235377,0.249795,0.235895,0.242188
Univ,0.613762,0.471188,0.615452,0.469824,0.739737,0.461890,0.738664,0.460423
Zara 1,0.448510,0.424574,0.451004,0.428294,0.540599,0.396331,0.537717,0.393949
Zara 2,0.421561,0.390705,0.422236,0.387588,0.484526,0.337074,0.483540,0.335329


In [40]:
df_best_of_20_ade.style.highlight_min(color = 'lightgreen', axis = 1)

,"No events, no grouping","Velocity event, no grouping","Angle event, no grouping","Angle + velocity event, no grouping","No events, with grouping","Velocity event, with grouping","Angle event, with grouping","Angle + velocity event, with grouping"
ETH,0.656323,0.470191,0.651449,0.469064,0.725268,0.544012,0.725567,0.546240
Hotel,0.128989,0.167158,0.131139,0.168486,0.134212,0.141257,0.135574,0.138327
Univ,0.306432,0.259277,0.306932,0.258522,0.356486,0.260324,0.356050,0.260046
Zara 1,0.230384,0.224434,0.231775,0.223946,0.263914,0.215832,0.263167,0.216248
Zara 2,0.210884,0.203598,0.211031,0.202142,0.234665,0.184283,0.234360,0.184238


In [42]:
df_single_fde.style.highlight_min(color = 'lightgreen', axis = 1)

,"No events, no grouping","Velocity event, no grouping","Angle event, no grouping","Angle + velocity event, no grouping","No events, with grouping","Velocity event, with grouping","Angle event, with grouping","Angle + velocity event, with grouping"
ETH,2.171929,2.171622,2.171786,2.171098,2.170491,2.178125,2.171380,2.173131
Hotel,0.474850,0.491318,0.473686,0.493402,0.474591,0.492834,0.475234,0.496542
Univ,1.231343,1.239020,1.231070,1.240064,1.229136,1.239504,1.229056,1.239307
Zara 1,1.003175,1.006727,1.002893,1.007137,1.001511,1.010094,1.001690,1.009235
Zara 2,0.778181,0.803550,0.778514,0.804929,0.777100,0.805967,0.777431,0.806225


In [45]:
df_single_ade.style.highlight_min(color = 'lightgreen', axis = 1)

,"No events, no grouping","Velocity event, no grouping","Angle event, no grouping","Angle + velocity event, no grouping","No events, with grouping","Velocity event, with grouping","Angle event, with grouping","Angle + velocity event, with grouping"
ETH,1.016617,1.015632,1.016943,1.016637,1.015722,1.019898,1.016231,1.017777
Hotel,0.246558,0.253870,0.245945,0.254224,0.246320,0.254581,0.246747,0.256266
Univ,0.581325,0.585112,0.581247,0.585539,0.579740,0.585303,0.579788,0.585106
Zara 1,0.469393,0.470761,0.469296,0.471469,0.468123,0.472473,0.468148,0.472147
Zara 2,0.365564,0.377115,0.365801,0.377809,0.364792,0.378106,0.364954,0.378297
